# Basic Statistics for Data Analysis

Notebook written by Sunmi Yoon (Data Analyst, Data Analysis Instructor)  
Contact: ysunmi0427@gmail.com, https://www.linkedin.com/in/yoonsunmi/

통계는 데이터를 다루는 수리적인 기법입니다. '평균'이라는 단어는 이걸 공부를 따로 해야하는건가 싶을 정도로 진부하지만, 데이터를 통해 나온 평균을 제대로 해석하기 위해서는 통계를 제대로 공부해야 합니다. 오늘 시간에는 평균, 분산, 표준편차와 같이 진부한 단어들을 새롭게 바라보는 일을 해보겠습니다. Box and Whisker Plot, Correlation heat map과 같이 기초 통계 시각화도 살펴봅시다. 통계의 모든 것을 가르치는게 아니라, 통계를 더 공부하고 싶게 만드는게 오늘의 목표입니다! 더불어 머리로 알고있는 통계 개념들을 코드로 옮기는 연습도 함께합시다.

*연습문제를 풀면서 따라오세요. (심화)가 붙어있는 문제는 시도해보고 어렵다면 건너뛰어도 무방합니다.*

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_moons, make_circles

import math
import statistics
from collections import Counter

In [2]:
# Please insure the version of matplotlib==3.1.0 for seaborn heatmap
# There is a little issue when drawing heatmap which cuts first and last low in half.
# If you have matplotlib version 3.1.1, please uninstall matplotlib and install version 3.1.0. Thanks!
# https://stackoverflow.com/questions/56942670/matplotlib-seaborn-first-and-last-row-cut-in-half-of-heatmap-plot

import matplotlib
print(matplotlib.__version__)

3.1.0


### Matplotlib 3.1.1을 가지고 계신 분들은
On Terminal,
1. `pip uninstall matplotlib` or `pip3 uninstall matplotlib` 
2. `pip install matplotlib==3.1.0` or use pip3

## 캐글 데이터 API로 데이터 다운로드 받기
0. `pip install kaggle` 또는 `pip3 install kaggle`
1. import kaggle로 OSError를 본다
2. kaggle 로그인 > My Account > Create New API Token을 클릭해 `kaggle.json` 파일을 다운로드 받는다.
3. OSError에서 알려준 디렉토리에 넣는다. Mac의 경우에는 command 명령어 `mv [현재파일위치] [옮기고싶은위치]`
4. `!kaggle competitions download -c titanic -p [원하는 다운로드 위치]`를 실행한다.

In [15]:
import kaggle

In [4]:
# !kaggle competitions download -c titanic -p ./ #느낌표는 터미널에서 원래 써야하는 명령어인데, pandas에서 넣으려고 했어 붙이는 거. 터미널에서 쓰려면 ! 빼야함 

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
titanic = pd.read_csv('./train.csv')

In [17]:
titanic.head(2) #survived 1이면 산거 sibsp 형제수 parch 가족 수 embarked 어떤 항구에서 탔는 지 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## 데이터의 출처
https://www.kaggle.com/c/titanic/data


* survival  
* pclass	    Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
* sex	
* Age	
* sibsp	# of siblings / spouses aboard the Titanic	
* parch	# of parents / children aboard the Titanic	
* ticket	Ticket number	
* fare	Passenger fare	
* cabin	Cabin number	
* embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

*더 자세한 Variable Notes는 링크에*

## 워밍업!

In [7]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
titanic.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [9]:
# Null 있는지 확인
titanic.isnull()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
5,False,False,False,False,False,True,False,False,False,False,True,False
6,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,True,False
8,False,False,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,False,True,False


In [10]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Summary 해석
* 데이터는 총 891명의 타이타닉 탑승자를 대상으로 합니다.
* Survived 컬럼의 평균이 0.3838이고, 살았을 때에 값은 1, 사망했을 때의 값은 0인 것으로 보아 데이터에 들어있는 사람 중 살아있는 사람은 38%입니다.
* Ticket의 등급은 1, 2, 3이고 평균이 2.30인 것을 보아 1등급보다는 2~3등급의 사람들이 더 많은 것으로 보입니다.
* 평균 나이는 29.69세이며 가장 어린 승객은 채 만 1살이 안 되었고(0.42), 최고령자는 80세였습니다.
* 운임은 평균 32였고 최저는 0, 최고는 512이며 std가 49.69인 것으로보아 승객들의 운임차이는 적지 않습니다.

### Q1. 실제로 Ticket의 등급은 어떤 분포를 합니까? 티켓 등급 별 승객수를 구하세요.

In [12]:
titanic.groupby(['Pclass']).size()

Pclass
1    216
2    184
3    491
dtype: int64

### Q2. 티켓 등급 별 사망자, 생존자를 구하세요. 티켓의 등급이 생존률에 영향을 미치나요?

In [47]:
titanic.groupby(['Pclass','Survived']).size() 
#titanic.groupby(['Pclass','Survived'])['PassengerId'].count() 이렇게 해줘도 됨(size) 안 쓰고 

Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
dtype: int64

In [52]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [61]:
titanic['gender']=titanic['Sex'].apply(lambda x:1 if x== 'male' else 0)
#sex type 값을 숫자형 변수로 변형 

# 오늘은(내일까지 일수도) 이런 것들을 새롭게 배웁니다.

1. Central Tendency 중심경향성
    - Mean 평균
    - Median 중앙값
    - Mode 최빈값
2. Measure of Variation 산포의 척도
    - Range, Interquartile Range 범위, 사분범위
    - Variance, Standard Deviation 분산, 표준편차
    - Coefficient of Variation 변동 계수
3. Box and Whisker Plot
4. Correlation 상관관계
5. Correlation and Causation 상관관계와 인과관계
6. 기초통계가 모든 것을 설명하지는 않는다
    1. 평균, 분산, 상관관계의 함정. [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe's_quartet)
    2. 상관관계의 함정
7. Simpson's Paradox 심슨의 역설

# 1. Central Tendency 중심경향성
데이터의 중심이 어디있는지를 나타내는 지표를 중심경향성(Central Tendency)라고 합니다. 대부분 평균(Average)를 사용하는데요, 필요에 따라 Median(중앙값), 최빈값(Mode)를 사용하기도 합니다. 평균에도 종류가 굉장히 많은데요. 산술평균, 기하평균, 조화평균, 가중산술평균에 대해서 들어보신적이 있을겁니다. 다양한 평균에 대해서는 『세상에서 가장 쉬운 통계학 입문』을 참고해주세요!

In [ ]:
a = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
# 산술평균
def mean(x):
    return sum(x) / len(x)

In [ ]:
mean(a)

산술평균의 문제: 산술평균의 데이터의 모든 값에 영향을 받는다는 특징이 있어요. 
그래서 이상한 값이 있으면 평균이 이동해버림

예를들어 리스트 a의 가장 큰 값을 100으로 바꿔보겠습니다.

In [ ]:
a[7] = 100
a

In [ ]:
mean(a)

이에 비교해 중앙값(Median)은 데이터의 모든 값에 영향을 받지는 않습니다. 리스트의 가장 큰 값이 1000으로 바뀐다고 하더라도, 기존의 중앙값을 유지합니다. 모든 데이터의 변화를 감지하여 유연하게 움직이는 평균과는 다릅니다.  
중앙값은 데이터에서 가장 중앙에 있는 포인트를 의미하며, 데이터 포인터가 짝수개라면 전체 데이터에서 가장 중앙에 있는 두 데이터 포인트의 산술평균을 의미합니다. 데이터 포인트가 짝수일때, 홀수일때 계산 방법이 다르므로 계산이 약간 복잡하죠? 중앙값을 구하는 함수 median()을 직접 구현해봅시다.

### Q3. 중앙값을 계산하는 함수를 구현하세요. (심화)

In [26]:
def median(x):
    '''
    x의 중앙값을 계산하는 함수를 구현하세요.
    if 조건문부터 세워야함/ 개수가 짝수면, 홀수면 그래서 case를 두개 나누어서 짜기 
    '''
    x = sorted(x) #리스트가 순서대로 sort됨 
    if len(x)%2 == 0:
        m = (x[len(x)//2-1]+x[len(x)//2])/2
        return m
    else:
        m = x[len(x)//2]
        return m

In [27]:
median([4,5,6,3])

4.5

In [28]:
# Q3을 위한 힌트

# 테스트를 위해 파이썬 내장라이브러리인 statistics의 median 함수를 활용
print(statistics.median([1, 2, 3]))
print(statistics.median([1, 2, 3, 4]))

# 리스트 정렬
print(sorted([3, 2, 1]))


# 나누기의 몫
print(4 // 2)

2
2.5
[1, 2, 3]
2


분위는 중앙값을 포함하는 개념으로 데이터를 크기 순서대로 정렬했을때 어디에 위치해 있는지를 뜻합니다. 중앙값은 50분위를 의미합니다. Pandas의 `describe()` 함수에서도 25%, 50%, 75% 들을 본 적이 있을겁니다. 여기에서 50%가 데이터를 순서대로 정렬했을 때 그 중앙, 중앙값을 의미합니다. 다른 방법으로도 간단하게 계산할 수 있습니다. 데이터의 이상치(Outlier: 지나치게 값이 크거나, 작거나, 대부분의 데이터들과는 패턴이 아주 다른 값)를 다룰 때에 `quantile()`함수를 자주 사용합니다.
데이터를 쭉 세웠을 때 20등인 애가 quantile(0.2) 이라고 생각 
quantile(0.99) 99등 
밑에 보면 90등에 있는 애가 나이가 50이다
이상한 값을 찾는 데 이 함수를 이용 
'quntile(1)' -> 맥시멈 값

In [29]:
print(titanic['Age'].quantile(0.1))
print(titanic['Age'].quantile(0.5))
print(titanic['Age'].quantile(0.9))

14.0
28.0
50.0


평균, 중앙값 이외에 최빈값(Mode)도 가끔 사용합니다. 데이터에서 가장 많이 등장하는 값을 의미하며 꼭 하나는 아닐 수 있습니다. 간단하게는 statistics의 `statistics.mode()`함수로 계산할 수 있습니다.

In [30]:
statistics.mode([1, 2, 3, 3, 4])

3

하지만 최빈값이 2개 이상일 때 `statistics.mode()` 함수는 에러를 출력합니다. `StatisticsError: no unique mode; found 2 equally common values`

In [31]:
# statistics.mode([1, 2, 3, 3, 4, 4])

### Q4. 최빈값이 2개 이상일 때에는 리스트를 출력하는 함수를 만들어 봅시다. (심화)

In [39]:
from collections import Counter
def mode(x):
    '''최빈값이 2개 이상일 때 리스트를 출력합니다'''
    l = []
    counter_x = Counter(x)
    max_count = max(counter_x.values())
    for k, v in Counter(x).items():
        if v == max_count:
            l.append(k)
    return l

In [40]:
# 수민님 코드 : 근데 이건 빈 배열[]이 들어가면 안 나와짐 (if 문을 수정하면 되지 않을까? 원래 mode에선 어떻게 처리할까? )
# -> 찾아보니까 원래 mode에서도 빈 배열[]은 error 처리 내줌 
def mode(x):
    cnt = Counter(x)
    # print(cnt)
    # print(cnt.items())
    # print(cnt.keys())
    # print(cnt.values())
    cnt_max = max(cnt.values())
    # print(cnt_max)
    mode = []
    for i in cnt:
        if cnt[i] == cnt_max:
            mode.append(i)
    if len(mode) == 1:
        return mode[0]
    else:
        return mode

In [41]:
# 테스트 
mode(['A', 'A', 'B', 'B', 'C'])

['A', 'B']

In [32]:
# Q4를 위한 힌트
ex = ['A', 'A', 'B', 'B', 'C']

# 리스트안에 있는 각 요소와, 요소의 갯수를 세서 딕셔너리 형태로 반환해주는 Counter 함수
cnt = Counter(ex)
print(cnt)

# Counter의 다양한 보조함수
print(cnt.values())
print(cnt.items())

Counter({'A': 2, 'B': 2, 'C': 1})
dict_values([2, 2, 1])
dict_items([('A', 2), ('B', 2), ('C', 1)])


In [35]:
Counter([1,1,2,2,3]) #key:v , k:v 이런식으로 나타내줌

Counter({1: 2, 2: 2, 3: 1})

In [36]:
max(Counter([1,1,2,2,3])).Count_valuse()

AttributeError: 'int' object has no attribute 'Count_valuse'

In [38]:
for k,v in Counter([1,1,2,2,3]).items():
    if v == max_count():
        l.append(k)
    return l

SyntaxError: 'return' outside function (<ipython-input-38-d9a46a75b95f>, line 4)

# 2. Measure of Variation 산포의 척도
데이터가 얼마나 퍼져있는지를 나타냅니다. 정규교육을 밟으면서 우리는 산포의 척도 중 분산과 표준편차를 배웠습니다. 나머지는 이보다도 더 쉽습니다. 직관적으로 생각해보면 데이터의 최대값에서 최소값을 뺀 것도 데이터가 얼마나 퍼져있는지를 나타내줄 수 있겠지요. 이를 `range`라고 정의할 수 있습니다.

In [ ]:
def data_range(x):
    return max(x) - min(x)

In [ ]:
data_range([1, 2, 3, 4, 100])

In [ ]:
titanic['Age'].max() - titanic['Age'].min()

비슷하게 데이터의 상위 25%에 해당하는 값과 하위 25%에 해당하는 값의 차이를 계산해 볼 수도 있겠지요. 이를 특별히 `interquartile range`라고 부릅니다. 타이타닉 데이터를 활용해볼까요? 타이타닉 탑승객 나이의 interquartile range는 아래와 같습니다. max와 min 값이 너무 극과 극이라 떨어져 있고 중앙에만 값들이 모여있을 경우에 range 범위가 이상하니까 interquartitle range를 구한다

In [ ]:
titanic['Age'].quantile(0.75) - titanic['Age'].quantile(0.25)

### Q5. 주어진 리스트의 interquartile range를 구하는 함수를 구현하세요.

In [ ]:
def interquartile_range(x):
    pass np.quantile(x,)

In [42]:
interquartile_range([1,2,3,4,5]_

SyntaxError: invalid syntax (<ipython-input-42-57bbfc93cb44>, line 1)

In [ ]:
# Q5를 위한 힌트

# 리스트의 quantile를 계산하는데 numpy의 함수를 활용하세요.
np.quantile([1, 2, 3, 4, 5], 0.25)

학창시절의 기억을 되살려서 분산과 표준편차를 구해볼까요. 간단하게는 numpy의 `np.var()`, `np.std()` 함수를 사용할 수 있지만 이 함수들을 사용하지 않고 구현해봅시다. Pandas의 `df.describe()` 함수의 결과에서도 표준편차를 볼 수 있습니다. `std`를 봐주세요. 표준편차 값이 클수록 데이터가 퍼져있음을 의미합니다.

In [ ]:
print(np.var([1, 2, 3, 4, 5]))
print(np.std([1, 2, 3, 4, 5]))

In [ ]:
titanic.describe()

### Q6. 분산을 계산하는 함수를 구현하세요.

In [ ]:
def variance(x):
    '''
    1. 리스트의 각 요소에서 평균을 뺀다. (편차) #평균에서 각 리스트가 얼마나 멀리 있는 지 알아보기 위해 그러면 막 - 값들도 나옴
    2. 평균을 뺀 값들을 제곱한다. (편차의 제곱) # - 도 있으니까 절대값으로 보고 싶기 때문에 제곱
    3. 편차의 제곱을 평균한다. (편차의 제곱의 평균) # 위의 절대값처럼 구한 제곱의 평균을 구함
    '''
    pass

In [ ]:
def variance(x):
    mean_x = np.mean(x)
    return 

In [43]:
a=[1,2,3]

In [62]:
empty = []
for i in a:
    empty.append(i+1)

In [63]:
empty

[2, 3, 4]

In [64]:
empty = [i+1 for i in a]

In [66]:
empty

[2, 3, 4]

In [76]:
mail = ['canary25@naver.com', 'monica4651@naver.com']

In [77]:
empty = []
for i in mail:
    empty.append(i.replace('@navercom',''))

In [75]:
empty

['canary25@naver.com', 'monica4651@naver.com']

In [ ]:
[i.replace('@naver.com','')]

In [78]:
# 문제 1 : a리스트 안에 있는 모든 원소를 제곱하여 a_squared에 저장하라 
a = [3,4,5,6,7]

In [80]:
a_squared = [i**2 for i in a]

In [81]:
a_squared

[9, 16, 25, 36, 49]

In [128]:
# 문제 2 : b 리스트 안에 있는 모든 원소들에서 b의 평균을 빼라 
b = [7,8,9,10,11]
# b의 평균은 numpy 함수를 사요하자 
np.mean(b)

9.0

In [129]:
# 이 방법은 계산이 비싸다 
b = [i-np.mean(b) for i in b]
# 계산 싸게하자 
mb = np.mean(b)
b = [i-mb for i in b]

In [130]:
b

[-2.0, -1.0, 0.0, 1.0, 2.0]

In [131]:
b - np.mean(b) # 이거는 돌아간다! (리스트에서 그냥 - 2 하면 안됨)
# 이건 numpy 기능인데 broadcasting 기능 

array([-2., -1.,  0.,  1.,  2.])

In [132]:
# 문제 3 : b 리스트의 각 요소에서 평균을 뺀 다음, 각 요소들을 제곱해보자 
b = [(i-mb)**2 for i in b]
b

[4.0, 1.0, 0.0, 1.0, 4.0]

In [133]:
# 문제 4 : b 리스트의 각 요소에서 평균을 뺀 다음, 각 요소들을 제곱 한 후 그 평균을 구하자
np.mean([(i-mb)**2 for i in b])

6.8

In [44]:
# 편차
diff = []
for i in a:
    diff.append(i-2)

In [ ]:
# 파이썬 comprehension 방식 
diff = [i-2 for i in a] #a리스트의 원소들을 하나씩 빼고 for 앞의 연산을 해서 i에 넣겠드 

In [45]:
# 편차의 제곱
diff_squared=[]
for i in diff:
    diff_squared.append(i**2)

In [46]:
# 편차의 제곱의 평균
sum(diff_squared)/len(diff_squared)

0.6666666666666666

In [ ]:
# Q6을 위한 힌트

# 리스트의 평균구하기
print(np.mean([1, 2, 3])) # numpy의 mean() 함수 활용
print(mean([1, 2, 3])) # 위에 구현한 mean() 함수 사용

# 파이썬에서 제곱하기
print(2**2)
print(3**2)

### Q7. 표준편차를 구현하세요.

In [134]:
def standard_deviation(x):
     # 위에 분산을 구했던 평균에서 -> 다시 root를 씌워서 제곱 했던 값을 다운 시킴 - 표준편차
    mean_x=np.mean(x)
    return math.sqrt(np.mean([(i-mean_x)**2 for i in x]))

In [ ]:
# 재민님 코드 
def standard_deviation(x):
    return math.sqrt(variance(x))

In [135]:
# Q7을 위한 힌트

# 제곱근 구하기
math.sqrt(4)

2.0

# comprehension 기능 몇 개 더 

In [137]:
a = [1,2,3,4]

In [138]:
# 리스트 a에 담긴 원소들 중에 짝수만 걸러서 리스트로 만들어라 

In [148]:
r = []
for i in a:
    if i%2 == 0:
        r.append(i)
r


[2, 4]

In [150]:
# comprehension with if filter
[i for i in a if i%2==0] #i가 짝수인 애들이 나올 때, i를 출력해주라는 뜻 -> a에 있는 리스트 부분만 나올 수 있구나~ -> 필터링 목적

[2, 4]

In [151]:
# 앞에서 if문 돌리는 방법  : comprehension with if which change value
# if 조건문을 사용하여 값을 바꿔주고 싶을 때 
['짝수' if i%2 == 0 else '홀수'for i in a]
# if로 값을 변환할 때 쓰는 방법 : 순서가 거꾸로  원래는 
# if i%2==0:
#     '짝수' -> 요런 형식인데 한줄로 써주는 거임

['홀수', '짝수', '홀수', '짝수']

In [ ]:
# 필터링을 쓰고 싶을 때와, 값을 바꿔주고 싶을 때를 구분 잘해서 쓰기 ! 

In [153]:
[i+1 for i in a] # 원래 우리가 쓰는 거 
[[i for i in a] for i in a] #리스트 컴프리헨션 안에 또 리스트 컴프리헨션

[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]

# dict comprehension

In [154]:
d =  {'apple': 500, 'banana': 300, 'carrot': 1000}

In [155]:
# k랑 v 순서를 바꿔주고 싶음 
{v:k for k,v in d.items()}

{500: 'apple', 300: 'banana', 1000: 'carrot'}

In [ ]:
# dict {}
# list []
# tuple ()

# 문제 
- 문자열 리스트가 주어졌을 때, 특정 길이의 원소만을 가지는 새로운 리스트를 반환하는 함수를 작성하라 

In [157]:
def filter_by_length(x, n):
    return [i for i in x if len(i) == n]

In [158]:
filter_by_length(['hi','mate'],2)

['hi']

# 문제2

In [168]:
price = {
    'tuna can': 2000,
    'sand': 10000,
    'biscuit': 5000
}
quantity = {
    'tuna can': 10,
    'sand': 1,
    'biscuit': 2
}

In [169]:
def net_order_value(price,quantity):
    return [i for k,v in ㅔ if sum(price)*sum(quantity)]

In [172]:
def net_order_value(price, quantity):
    return sum({k:v * quantity[k] for k, v in price.items()}.values())

# 3. Box and Whisker Plot
* Box and Whisker Plot의 개념 설명에 대해서 잘 설명된 글: https://boxnwhis.kr/2019/02/19/boxplot.html
* [Pandas로 Box and Whisker Plot 그리기 official manual](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html)

### Q8. 나이는 생존에 영향이 있었을까요? Pandas groupby 함수를 사용해서 알아보세요.

In [ ]:
# Answer

## Box and Whisker Plot with Pandas
위에서 배웠던 내용들을 한번에 시각화 할 수 있음

In [ ]:
titanic.boxplot(#가로 안에서 shft+tap 누르면 여기 안에 써야하는 정보를 볼 수 있음/ iqr:interquntile range)

In [ ]:
titanic.boxplot(column=['Age'], by='Survived', figsize=(15,6), vert=False)
# 15: 가로 길이 6: 세로 길이 vert(vertical): 설명안하면 세로로 만들어짐. 근데 가로로 배워서 가로로 그린 게 좋아서 false로 함)

### Q9. Box and Whisker Plot의 결과를 해석해보세요.

# 4. Correlation 상관관계

두 변수 사이에 어떤 관계(Correlation)가 있는지 알아보기 위해서 우리는 공분산, 상관계수같은 개념을 이용합니다. 상관관계는 공분산을 X, Y 각각의 표준편차를 나눠 계산하며 -1(완벽한 음의 상관관계)에서 1(완벽한 양의 상관관계) 사이의 값을 가집니다. 상관관계가 0이라면 X, Y간의 그 어떤 양의 또는 음의 상관관계가 없음을 의미합니다. 만약 상관관계가 0.2라면 상대적으로 약한 양의 상관관계를 가진다 말 할 수 있습니다.
* Mathematical notation: https://en.wikipedia.org/wiki/List_of_mathematical_symbols_by_subject

### Q10. 공분산을 계산하는 함수 `covariance()`를 구현하세요.

\begin{align}
\sigma(X, Y) = \dfrac{1}{N} \sum_{i=1}^{N} (x_i - \bar{x}) (y_i - \bar{y}) 
\end{align}

In [ ]:
# 공분산 : x와 y의 크기에 따라 영향을 많이 받아서, 아까 root를 씌워준 것처럼 상관계수를 구해준다
def covariance(x, y):
    pass

In [ ]:
# 10번 문제를 쉽게 풀기위한 힌트

# numpy의 dot() 함수는 각 성분별로 곱한 값을 더해줍니다. 
np.dot([1, 2], [1, 2]) # (1*1 + 2*2 = 5)

### Q11. 상관계수를 계산하는 함수 `correlation()`를 구현하세요.

\begin{align}
\rho(X, Y) = \dfrac{\sigma(X, Y)}{\sigma(X) \sigma(Y)}
\end{align}

In [ ]:
# 공분산 : x와 y의 크기에 따라 영향을 많이 받아서, 아까 root를 씌워준 것처럼 상관계수를 구해준다
def correlation(x, y):
    pass

Pandas에서는 데이터프레임 전체의 관계를 한 눈에 살펴볼 수 있도록 `corr()` 함수를 제공합니다.

In [ ]:
titanic.corr() #상관관계가 1이 나옴 -> 비례 

seaborn이라는 시각화 라이브러리를 이용해 heatmap을 그려 볼 수도 있습니다.

In [ ]:
sns.heatmap(titanic.drop('PassengerId', axis=1).corr(), annot=True, fmt = '.2f', linewidths=0.2)
plt.show()
#.drop: 특정 컬럼을 없애는 함수. 그래서 passengerid라는 칼럼을 없애줘! axis하면 열을 없애고 행을 없애려면 row 
# 그리고 .corr() 상관함수를 만들어주고 annot=true -> 숫자 표시해줘 
# fmt = 소수점 보여줘 (.2면 두자리수까지)
# linewidths : 사각형들 사이의 간격 


### Q12. Correlation matrix를 해석해보세요.
참고자료: https://www.slideshare.net/ssuser2fe594/2107-80754131 p56부터

* Answer
* Answer

## 이상치를 포함한 상관관계

In [ ]:
anscombe1_outlier = pd.DataFrame({
    'x': [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5, 17],
    'y': [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68, 0.5]
})

In [ ]:
sns.lmplot(data=anscombe1_outlier, x='x', y='y', ci=None)

이상치를 제거하니 더 강력한 상관관계를 볼 수 있다. 상관관계 분석에서 이상치를 발견하고 제거해주는 것이 이래서 중요하다.

In [ ]:
sns.lmplot(data=anscombe1_outlier.drop(index=11), x='x', y='y', ci=None)

### Q13. Titanic 데이터에서 이상치를 관찰하고, 이상치를 제외한 상관계수를 다시 계산해봅시다.

# 5. Correlation and Causation 상관관계와 인과관계

![Correlation is not Causation](http://commons.trincoll.edu/cssp/wp-content/blogs.dir/68/files/2013/12/CorrelationCausationFinal1.jpg)

# 6. 기초통계가 모든 것을 설명하지는 않는다.

## 1. 평균, 분산, 상관관계의 함정. [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe's_quartet)

In [ ]:
anscombe1 = pd.DataFrame({
    'x': [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5],
    'y': [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68],
    'group': 1
})

anscombe2 = pd.DataFrame({
    'x': [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5],
    'y': [9.14, 8.14, 8.74, 8.77, 9.26, 8.1, 6.13, 3.1, 9.13, 7.26, 4.74],
    'group': 2
})

anscombe3 = pd.DataFrame({
    'x': [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5],
    'y': [7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73],
    'group': 3
})

anscombe4 = pd.DataFrame({
    'x': [8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8],
    'y': [6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.5, 5.56, 7.91, 6.89],
    'group': 4
})

In [ ]:
anscombes_quartet = pd.concat([anscombe1, anscombe2, anscombe3, anscombe4])

In [ ]:
anscombes_quartet.groupby(['group']).describe()

In [ ]:
anscombes_quartet.groupby(['group']).corr()

In [ ]:
sns.lmplot(x='x', y='y', col='group', hue='group', data=anscombes_quartet, fit_reg=True, ci=None, palette="husl")

`Anscombe's quartet`은 평균, 분산, 상관관계가 거의 동일한 네 데이터셋이다. 대표적인 요약통계량들이 거의 일치함에도 데이터의 생김새는 매우 다르다. 이 네 데이터셋을 같은 패턴을 보이는 데이터라고 설명할 수 있을까?

## 2. 상관관계의 함정

In [ ]:
df = pd.DataFrame({
    'x': [-1, -2, 0, 1, 2],
    'y': [1, 2, 0, 1, 2]
})

In [ ]:
df

In [ ]:
np.corrcoef(df['x'], df['y'])

x와 y의 상관계수는 0입니다. 그렇다고 두 변수간에 아무런 관계도 존재하지 않는 것일까요? 아닙니다. y는 x의 절대값이라는 선명한 관계가 있습니다. 이처럼 상관계수로 모든 관계를 설명할 수는 없습니다.

In [ ]:
df.plot.scatter(x='x', y='y')

## 상관관계의 함정, 또 다른 예시

In [ ]:
circle = make_circles(n_samples=100, shuffle=True, noise=None, random_state=None, factor=0.999)

In [ ]:
circle_df = pd.DataFrame({
    'x': [c[0] for c, i in zip(circle[0], circle[1]) if i == 0],
    'y': [c[1] for c, i in zip(circle[0], circle[1]) if i == 0]
})

In [ ]:
circle_df.plot.scatter(x='x', y='y')

In [ ]:
np.corrcoef(circle_df['x'], circle_df['y'])

# 7. Simpson's Paradox 심슨의 역설

## Simpson's Paradox in Titanic dataset
* Age & Fare (confounding variable: class)
* Class & Survival rate (confounding variable: gender)

## References
### Books
* 『밑바닥부터 시작하는 데이터 과학』
* 『세상에서 가장 쉬운 통계학 입문』

### About Box and Whisker Plot 
* https://boxnwhis.kr/2019/02/19/boxplot.html

### About Simpson's Paradox
* https://stats.stackexchange.com/questions/185047/is-this-simpsons-paradox-on-the-titanic-data-set
* https://select-statistics.co.uk/blog/hidden-data-and-surviving-a-sinking-ship-simpsons-paradox/
* https://www2.stat.duke.edu/courses/Fall12/sta611/SimpsonsParadox.pdf